# NMT with Transformer&BPE
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2020/blob/master/08-nlp-part2/NMT-Transformers-bpe.ipynb)

**Цели тетрадки**

- Познакомитсья с BPE (будем пользоваться пакетом youtokentome)
- Освоить работу с nn.Transformer
- Натренировать модель для переводов en <--> ru


**План**

- Предобработать данные и обучить BPE-модель
- Написать обвязку вокруг nn.Transformer для задачи перевода
- Натренировать и проверить модель


**Настоятельно рекомендуется воспользоваться колабом**

In [ ]:
# установим отсутствующие библиотеки:
! pip install youtokentome

NMT с помощью трансформеров и BPE представлений.
Начинаем точно так же с датасета:


Для преобразования текстов в BPE и обратно воспользуемся библиотекой
https://github.com/VKCOM/YouTokenToMe

Она быстро работает, позволяет хранить данные в текстовом формате и преобразовывать в BPE (с BPE-дропаутом) прямо на лету.

In [ ]:
%matplotlib inline
from IPython.display import clear_output
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm.auto import tqdm
import unicodedata

import youtokentome as yttm

## Данные

Работать будем с тем же датасетом en+ru с сайта открытого проекта tatoeba.
Но на этот раз в качестве будем представлять данные не в виде слов, а в виде sub-word частей, с помощью Byte Pair Encoding.

Для работы с BPE возьмем библиотеку [YouTokenToMe](https://github.com/VKCOM/YouTokenToMe). 

Она работает быстро, это позволит нам хранить данные в текстовом формате, и делать все необходимые преобразования прямо на лету.

In [ ]:
! wget -qO- https://github.com/m12sl/dl_cshse_2019/raw/master/seminars/x2seq/eng-rus.tar.gz | tar xzvf -

In [ ]:
# Приготовим данные и посмотрим на них
# Кроме словаря нас интересует еще набор символов
raw_alphabet = set()
alphabet = set()
def normalize(s):
    return "".join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess(s):
    raw_alphabet.update(s)
    s = normalize(s.lower().strip())
    s = re.sub(r"[^a-zа-я?.,!]+", " ", s)
    s = re.sub(r"([.!?])", r" \1", s)
    alphabet.update(s)
    return s

pairs = []
with open('eng-rus.txt', 'r') as fin:
    for line in tqdm(fin.readlines()):
        pair = [preprocess(_) for _ in line.split('\t')]
        pairs.append(pair)

print("RAW alphabet {} symbols:".format(len(raw_alphabet)), 
      "".join(sorted(raw_alphabet)))
print("After preprocessing {} symbols: ".format(len(alphabet)), 
      "".join(sorted(alphabet)))
print("There are {} pairs".format(len(pairs)))
print(pairs[10101])

## обучение BPE

BPE позволяет нам проучить словарь произвольных размеров. 

Например, мы можем сделать общий словарь для английского и русского.
Для этого надо записать все доступные тексты в один файл и проучить BPE.

In [ ]:
lines = []
for p in pairs:
    lines += p
lines = list(set(lines))
with open("./all.txt", "w") as fout:
    for line in lines:
        fout.write(line + "\n")

! head all.txt

In [ ]:
VOCAB = 5000
bpe = yttm.BPE.train(data="./all.txt", vocab_size=VOCAB, model="enru.bpe")

## NB: можно ли учить BPE на всем датасете

Во многих задачах возникает вопрос касательно расчета статистик на всем датасете:

<mark>если _что-то_ является важным признаком, нужно это _что-то_ считать только по трейну, или можно взять весь датасет с валидацией?</mark>

- можно ли считать средние по всем доступным данным для задачи прогнозирования временных рядов
- можно ли считать word2vec на всем датасете
- и т.д.

Простого ответа нет, в данном случае BPE не является в прямом смысле моделью, но изменение статистик может влиять на состав словаря:

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/bpe.gif" crossorigin="anonymous"/>

In [ ]:
bpe.encode(lines[0])

In [ ]:
print(bpe.encode(lines[:10], output_type=yttm.OutputType.ID))
print(bpe.encode(lines[:10], output_type=yttm.OutputType.SUBWORD))

## BPE Dropout

(см статью: [BPE-Dropout: Simple and Effective Subword Regularization](https://arxiv.org/abs/1910.13267))

В очень больших BPE словарях (5к токенов на два языка -- это небольшой словарь) встречается проблема: некоторые токены есть в словаре, но не встречаются в трейне. 

Они могут встречаться в реальных данных, благодаря естественным процессам или опечаткам. Для борьбы с таким явлением и просто в качестве регуляризации можно применить BPE-dropout: случайное переразбиение строки на токены.
<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/bpe-dropout.png" crossorigin="anonymous"/>

    

In [ ]:
print(bpe.encode(lines[:1], dropout_prob=0.0, output_type=yttm.OutputType.SUBWORD))
print(bpe.encode(lines[:1], dropout_prob=0.2, output_type=yttm.OutputType.SUBWORD))
print(bpe.encode(lines[:1], dropout_prob=0.5, output_type=yttm.OutputType.SUBWORD))

In [ ]:
encoded0 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.0)]
encoded1 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.1)]
encoded2 = [len(_) for _ in bpe.encode(lines, dropout_prob=0.5)]

sns.distplot(encoded0, kde=False, label="no do")
sns.distplot(encoded1, kde=False, label="do 0.1")
sns.distplot(encoded2, kde=False, label="do 0.5")
plt.legend()
plt.yscale('log')

In [ ]:
# предлагается максимальную длину строки ограничить 100 токенами, а для обучения использовать BPE_DO=0.1
MAX_LENGTH = 100

## Dataset

Датасет в этот раз минимально простой: возвращает словарь с en и ru строками, без преобразований.

collate_fn нам не потребуется, а конвертацию в BPE мы опишем внутри класса модели.

In [ ]:
class Pairset:
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        en, ru = self.data[item]
        return dict(en=en, ru=ru)

train_pairs, val_pairs = train_test_split(pairs, test_size=0.3)

trainset = Pairset(train_pairs)
valset = Pairset(val_pairs)

In [ ]:
trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
it = iter(trainloader)
print(next(it))

## Train Loop

Тренировочный цикл обычный. Подразумевается, что у модели есть два метода:

```
model.compute_all(batch) -> Dict
model.check_translations(batch) -> None
```

In [ ]:
def train_model(model, opt, trainloader, valloader, epochs=1):
    step = 0
    logs = defaultdict(list)
    for epoch in range(epochs):
        model.train()
        for batch in tqdm(trainloader):
            details = model.compute_all(batch)
            loss = details["loss"]
            opt.zero_grad()
            loss.backward()
            opt.step()
            step += 1
            [logs[k].append(v) for k, v in details["metrics"].items()]
            
        model.eval()
        tmp = defaultdict(list)
        
        with torch.no_grad():
            for batch in tqdm(valloader):
                details = model.compute_all(batch)
                for k, v in details["metrics"].items():
                    tmp[k].append(v)
            tmp = {k: np.mean(v) for k, v in tmp.items()}
            [logs[f"val_{k}"].append(v) for k, v in tmp.items()]
            logs["step"].append(step)
            model.check_translations(batch)
        
        for key in ["loss"]:
            plt.figure()
            plt.title(key)
            plt.plot(logs[key], label="train", c='b', zorder=1)
            plt.scatter(logs["step"], logs[f"val_{key}"], label="val", c='r', zorder=10)
            plt.legend()
            plt.grid()
            plt.show()

## Задачи и Модели

В прошлый раз мы делали RNN Encoder-Decoder модель, которая переводила с одного языка на другой.

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/nmt-encoder-decoder.png" crossorigin="anonymous"/>

Какие есть варианты, как еще можно сделать перевод?

**NMT как Language Model (или как автодополнение)**

Мы можем рассматривать перевод как задачу продолжения текста и учить как обычную одностороннюю языковую модель.

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/nmt-lm.png" crossorigin="anonymous"/>


Что, если мы хотим переводить с множества языков на множество других?

Простейший вариант: тренировать отдельные модели на каждую пару языков.

Более интересный вариант: для каждого языка тренировать энкодер в общее представление и декодер из него.

И один из наиболее интересных вариантов: делать переводы в рамках одной модели.

**NMT как Text-to-Text**

Можно сказать модели какой перевод нужно сделать, например с помощью специального токена в энкодер:

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/nmt-cmd-1.png" crossorigin="anonymous"/>

Специальный токен можно подать в качестве первого токена в декодер:

<img src="https://github.com/m12sl/dl-hse-2020/raw/master/08-nlp-part2/img/nmt-cmd-2.png" crossorigin="anonymous"/>



Разумеется можно сделать иначе, сделать у сети отдельный вход для флажка, но подход с указанием задачи в тексте выглядит изящнее всего.


## One Model To Rule Them All
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b7/Unico_Anello.png" crossorigin="anonymous"/>


Пободный подход используют в [модели T5](https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html): модель учат решать разные задачи, задавая их токенами в SRC:
<img src="https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png" crossorigin="anonymous"/>


## `nn.Transformer`

Официальная документация по (nn.Transformer)[https://pytorch.org/docs/master/generated/torch.nn.Transformer.html#transformer] довольно скудная.

Но важные моменты таковы:

0. Входные и выходные данные надо готовить самостоятельно: понадобится написать positional и token embeddings, а также выходной FC-слой

1. nn.Transformer.forward берет на себя прогон энкодера, и правильное применение декодера.

2. Порядок осей такой же, как при использовании RNN-моделей (для совместимости в seq2seq задачах): `[seq_len, batch_size, dimension]`.

3. Обязательно надо задавать `src_key_padding_mask` и `tgt_key_padding_mask` для маскирования недоступных для внимания токенов (в частности паддингов).



Предлагается завести два спецтокена: для перевода в русский язык и в английский, с номерами `bpe.vocab_size()` и `bpe.vocab_size() + 1`. 
Эти токены можно не генерировать с помощью выходного слоя, но на входе они могут быть.



Предлагается написать следующие функции:

```
model.encode(list_of_strings) # функция, которая переводит строку в последовательность номеров BPE-токенов, добавляет спецтокены и паддит до MAX_LENGTH
```


```
model.check_translations(batch) # функция, которая сделает и выведет перевод для батча с примерами
```


```
model.compute_all(batch) # функция для обучения, прогонит батч, посчитает лосс и вернет словарь с метриками и лоссом
```

In [ ]:
class VeryT(nn.Module):
    def __init__(self, bpe, bpe_dropout=0.1, hidden_size=256):
        super().__init__()
        self.hidden_size = hidden_size
        self.bpe = bpe
        self.bpe_dropout = bpe_dropout
        self.embeddings = nn.Embedding(bpe.vocab_size() + 2, hidden_size)
        self.positional_embeddings = nn.Embedding(MAX_LENGTH, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size, 
            nhead=8, 
            num_encoder_layers=3, 
            num_decoder_layers=3, dim_feedforward=512)
        
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, bpe.vocab_size()),
            nn.LogSoftmax(dim=-1),
        )
        
        
    def encode(self, lst, pre=None, post=None, seq_len=None, dropout=0.0):
        lst = [self.bpe.encode(entry, dropout_prob=dropout) for entry in lst]
        ## add tokens and paddings
        <your code>
        return lst
        
    def check_translations(self, batch):
        en, ru = [batch[key] for key in ["en", "ru"]]
        src = self.encode(en, ...)
        dst = self.encode(ru, ...)
        <your code if needed>
        src = torch.LongTensor(src)
        dst = torch.LongTensor(dst)
        with torch.no_grad():
            # generate ouput autoregressively
            for i in range(10):  # MAX_LEN - 1
                <your code>
            dst = dst.cpu().numpy()
            dst = [line.tolist() for line in dst]
            dst = self.bpe.decode(dst)
            dst = [line.replace("<PAD>", "") for line in dst]
        for line in zip(en, ru, dst):
            print("\t".join(line))
    
    def compute_all(self, batch):
        en, ru = [batch[key] for key in ["en", "ru"]]
        <formulate task>
        src = self.encode(en, ...)
        dst = self.encode(ru, ... )
        
        src = torch.LongTensor(src)
        dst = torch.LongTensor(dst)

        output = self.forward(src, dst)
        
        <compute loss>
        loss = ...
        
        return dict(
            loss=loss,
            metrics=dict(
                loss=loss.item(),
            )
        )
        
        
    def forward(self, src, dst):
        # let's a little hack:
        device = next(self.parameters()).device
        src = src.to(device)
        dst = dst.to(device)
        
        
        <build embeddings for tokens and positional>
        
        # embedded = embedded_tokens * sqrt(hidden_size) + embedded_positions
        
        <reshape properly>
        
        <build pad masks>
        src_pad_mask = src != 0
        dst_pad_mask = dst != 0
        
        output = self.transformer(src_embedded, dst_embedded, 
                                  src_key_padding_mask=src_pad_mask, 
                                  tgt_key_padding_mask=dst_pad_mask)
        <predict next token probs>
        <permute to [bs, vocab, seq_len]> 
        return output

In [ ]:
# проверяем, размерности
model = VeryT(bpe)
with torch.no_grad():
    batch = next(it)
    model.check_translations(batch)

In [ ]:
if torch.cuda.is_available():
    device="cuda:0"
print(device)

model = VeryT(bpe)
model.to(device)
opt = optim.Adam(model.parameters(), lr=3e-4)


In [ ]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True)
valloader = DataLoader(valset, batch_size=50, shuffle=False)

train_model(model, opt, trainloader, valloader)